## **Loca Como Tu Madre**

In [1]:
from IPython import display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns; sns.set()
import datetime
import nbformat
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode (connected = True )
import plotly.graph_objs as go
import plotly.offline as py
from matplotlib.backends.backend_pdf import PdfPages
from fpdf import FPDF
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import locale
import plotly.io as pio

In [3]:
# 1) git log --pretty=format:%cI,%h,%an,%ae,%s > /home/claudio/@LCTM/commits_LCTM.csv
# 2) abrir excel e e na aba dados, importar o arquivo gerado pelo código acima


In [64]:
# setar locale para português para que os dias da semana retornem em portugês no código abaixo.
locale.setlocale(locale.LC_ALL, 'pt_BR.utf-8')

# Importing each csv file and then joinning then, 
# as the files for other months are send by the client we can insert each one at the end of this code

df1 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_07.csv', encoding="ISO-8859-1", sep = ';')
df2 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_08.csv', encoding="ISO-8859-1", sep = ';')
df3 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_09.csv', encoding="ISO-8859-1", sep = ';')
df4 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_10.csv', encoding="ISO-8859-1", sep = ';')
df5 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_11.csv', encoding="ISO-8859-1", sep = ';')
df6 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2022_12.csv', encoding="ISO-8859-1", sep = ';')
df7 = pd.read_csv(r'/home/claudio/@LCTM/lctm001/2023_01.csv', encoding="ISO-8859-1", sep = ';')

In [65]:
transactions = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

In [66]:
del transactions['Vr. Desconto']
del transactions['Valor Venda']
del transactions['Tx Serviço']
del transactions["Caixa"]
del transactions['Operador']
del transactions['Data Cancelamento']
del transactions['Operador Cancelamento']
del transactions['Operador Desconto']
del transactions['Operador Desc Repique']
del transactions['NRC']
del transactions['Comandas']
del transactions['Série NFCE']
del transactions['Série NFE']
del transactions['Integração']
del transactions['Observação']
del transactions['CO_CLIENTE']
del transactions['Atendente']
del transactions['IC_REVENDA']

In [97]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46465 entries, 1 to 46471
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   prim_col          4831 non-null   float64
 1   NFe               41635 non-null  object 
 2   NFCe              46463 non-null  object 
 3   Data Venda        46465 non-null  object 
 4   Data Recebimento  46465 non-null  object 
 5   Nome Cliente      41643 non-null  object 
 6   Valor Recebido    46465 non-null  object 
 7   Operação          46463 non-null  object 
 8   venda             46465 non-null  int64  
 9   Vendedor          4830 non-null   object 
 10  data_venda        46465 non-null  object 
 11  data_recebimento  46465 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 4.6+ MB


In [98]:
transactions.head()

,prim_col,NFe,NFCe,Data Venda,Data Recebimento,Nome Cliente,Valor Recebido,Operação,venda,Vendedor,data_venda,data_recebimento
1,981201.0,NaN,6490,30/07/2022 20:02:56,30/07/2022 22:58:00,NaN,209,1,981201,JESSICA,30/07/2022 20:02:56,30/07/2022 22:58:00
2,NaN,421,BURRATA COM JAMON E VERDE,1,49,49,30/07/2022 20:03:12,JESSICA,981201,NaN,nd,nd
3,NaN,41,PORÇAO NACHOS,1,15,15,30/07/2022 20:17:28,JESSICA,981201,NaN,nd,nd
4,NaN,97,JARRA SANGRIA LOCA SUMMER,1,55,55,30/07/2022 20:22:20,GABRIEL,981201,NaN,nd,nd
5,NaN,406,LULAS EMPANADAS,1,38,38,30/07/2022 21:22:51,JESSICA,981201,NaN,nd,nd


In [68]:
# renomeei a coluna 'Tipo Operação' para 'venda'.
transactions.rename(columns = {'Tipo Operação':'venda'}, inplace = True)

In [69]:
# Pandas Drop Rows Based on Column Value
# https://sparkbyexamples.com/pandas/pandas-delete-rows-based-on-column-value/
transactions.drop(transactions[transactions['venda'] == 'CO_VENDA'].index, inplace = True)

In [70]:
# renomeei a coluna 'Cod Venda' para 'prim_col'.
transactions.rename(columns = {'Cod Venda':'prim_col'}, inplace = True)

In [90]:
# If 'prim_col' value is not empty, set the value of column 'venda' to the value of column 'prim_col', else value of column 'venda' stays the same
transactions['venda'] = transactions.apply(lambda row: row['prim_col'] if row['venda'] == 'LOCA' else row['venda'], axis=1)

# If 'prim_col' value is not empty, set the value of column 'venda' to the value of column 'prim_col', else value of column 'venda' stays the same
transactions['venda'] = transactions.apply(lambda row: row['prim_col'] if row['venda'] == 'Balcao' else row['venda'], axis=1)


# Vamos analisar cada parte dessa linha:
#transactions['venda']: Estamos nos referindo à coluna 'venda' do DataFrame transactions.
# Essa coluna será atualizada com base no resultado da operação que será realizada a seguir.
#transactions.apply(...): O método apply() é usado para aplicar uma função a todas as linhas 
# (ou colunas) de um DataFrame. Neste caso, a função está sendo aplicada às linhas, conforme
# especificado pelo argumento axis=1.
#lambda row: ...: Estamos usando uma função lambda, que é uma função anônima e de uso único.
# Essa função aceita um argumento chamado row, que representa uma linha do DataFrame transactions.
#row['prim_col'] if row['venda'] == 'LOCA' else row['venda']: Esta é a lógica da função lambda.
# Estamos usando uma expressão condicional para verificar se o valor da coluna 'venda' na
# linha atual (row['venda']) é igual a 'LOCA'. Se for verdade, o valor da coluna 'prim_col'
# na mesma linha (row['prim_col']) será usado como resultado. Caso contrário, o próprio valor da coluna 'venda' será mantido.

In [94]:
# Função para verificar se o valor é uma string e contém apenas letras, testando se o código da celula anterior funcionou
def is_string_with_letters(value):
    return isinstance(value, str) and value.isalpha()

# Aplicando o filtro para manter apenas as linhas onde o valor da coluna 'venda' é uma string e contém apenas letras
filtered_transactions = transactions[transactions['venda'].apply(is_string_with_letters)]

# filtered_transactions

,prim_col,NFe,NFCe,Data Venda,Data Recebimento,Nome Cliente,Valor Recebido,Operação,venda,Vendedor,data_venda,data_recebimento


In [83]:
# Função para verificar se o valor é uma data
def is_date(value):
    try:
        pd.to_datetime(value)
        return True
    except ValueError:
        return False

# Função para aplicar à coluna
def copy_date_or_set_nd(value):
    return value if is_date(value) else 'nd'


In [84]:

# Criar a nova coluna 'data_venda' aplicando a função copy_date_or_set_nd
transactions['data_venda'] = transactions['Data Venda'].apply(copy_date_or_set_nd)



In [85]:
# Criar a nova coluna 'data_recebimento' aplicando a função copy_date_or_set_nd
transactions['data_recebimento'] = transactions['Data Recebimento'].apply(copy_date_or_set_nd)


In [96]:
# Convert the "venda" variable from object type to integer type
transactions['venda'] = np.int64(transactions['venda'])